In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = "/home/gbotemi/credentials/credentials.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar,./lib/spark-3.2-bigquery-0.29.0-preview.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/03/09 10:56:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df = spark.read.json('https://data.gharchive.org/2015-01-01-15.json.gz')

Py4JJavaError: An error occurred while calling o118.json.
: java.lang.UnsupportedOperationException
	at org.apache.hadoop.fs.http.AbstractHttpFileSystem.listStatus(AbstractHttpFileSystem.java:95)
	at org.apache.hadoop.fs.http.HttpsFileSystem.listStatus(HttpsFileSystem.java:23)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.util.

In [10]:
df = spark.read.parquet("gs://gharchive-data/pq/2015/*/*/*")

In [11]:
print(df.count())

2518370


In [12]:
df.show(1)

+----------+---------+--------------------+--------------------+--------------------+------+-------------------+----+
|        id|     type|               actor|                repo|             payload|public|         created_at| org|
+----------+---------+--------------------+--------------------+--------------------+------+-------------------+----+
|2542362445|PushEvent|{https://avatars....|{28595460, davans...|{null, 627627f750...|  true|2015-01-28 16:00:00|null|
+----------+---------+--------------------+--------------------+--------------------+------+-------------------+----+
only showing top 1 row



In [13]:
from pyspark.sql import types
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col, split, when

In [14]:
df_merge = df.withColumn("users" ,col("actor.login")) \
    .withColumn("repo_id", col("repo.id")) \
    .withColumn("repo_name", split(col("repo.name"), "/")[1]) \
    .withColumn("org_exists", col("org").isNotNull()) \
    .withColumn("count_commits", when(col("payload.size") >= 1, col("payload.size")).otherwise(0))

df_merge = df_merge.drop(col('actor')) \
    .drop(col('repo')) \
    .drop(col('payload')) \
    .drop(col('org'))

In [15]:
df_merge.show(1)

+----------+---------+------+-------------------+-------+--------+---------+----------+-------------+
|        id|     type|public|         created_at|  users| repo_id|repo_name|org_exists|count_commits|
+----------+---------+------+-------------------+-------+--------+---------+----------+-------------+
|2542362445|PushEvent|  true|2015-01-28 16:00:00|davansy|28595460|      pbb|     false|            1|
+----------+---------+------+-------------------+-------+--------+---------+----------+-------------+
only showing top 1 row



In [19]:
df_merge = df_merge.repartition("created_at")

In [20]:
df_merge.show(10)

+----------+-----------------+------+-------------------+-------------+--------+--------------------+----------+-------------+
|        id|             type|public|         created_at|        users| repo_id|           repo_name|org_exists|count_commits|
+----------+-----------------+------+-------------------+-------------+--------+--------------------+----------+-------------+
|2542373253|        PushEvent|  true|2015-01-28 16:03:26|        mmaid|27592699|       ingrid-portal|      true|            1|
|2542373293|        PushEvent|  true|2015-01-28 16:03:26|        kinjo|27030126|  my-ansible-project|     false|            1|
|2542373308|        PushEvent|  true|2015-01-28 16:03:26|   vschmidt94|29882861|               CS325|     false|            1|
|2542373316|        PushEvent|  true|2015-01-28 16:03:26|    greatfire|15100395|                wiki|     false|            1|
|2542373317|IssueCommentEvent|  true|2015-01-28 16:03:26|       MAT978|28109524| user-interface-text|      true

In [25]:
df_merge.write \
    .format("bigquery") \
    .mode("append") \
    .option("writeMethod", "direct") \
    .option("partitionType", "MONTH") \
    .option("partitionField", "created_at") \
    .save("gharchive-379414.gharchive_data.qwel")

In [4]:
!gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar lib/gcs-connector-hadoop3-2.2.5.jar

Copying gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar...
- [1 files][ 30.1 MiB/ 30.1 MiB]                                                
Operation completed over 1 objects/30.1 MiB.                                     


In [6]:
!gsutil cp gs://spark-lib/bigquery/spark-3.1-bigquery-0.29.0-preview.jar lib/spark-3.1-bigquery-0.29.0-preview.jar

Copying gs://spark-lib/bigquery/spark-3.1-bigquery-0.29.0-preview.jar...
- [1 files][ 36.3 MiB/ 36.3 MiB]                                                
Operation completed over 1 objects/36.3 MiB.                                     


In [33]:
!gsutil cp gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.13-0.29.0.jar lib/spark-bigquery-with-dependencies_2.13-0.29.0.jar

Copying gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.13-0.29.0.jar...
/ [1 files][ 36.3 MiB/ 36.3 MiB]                                                
Operation completed over 1 objects/36.3 MiB.                                     


In [9]:
!gsutil cp gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar lib/spark-bigquery-latest_2.12.jar

Copying gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar...
- [1 files][ 33.9 MiB/ 33.9 MiB]                                                
Operation completed over 1 objects/33.9 MiB.                                     


In [14]:
!ls lib

gcs-connector-hadoop3-2.2.5.jar  spark-3.2-bigquery-0.29.0-preview.jar


In [2]:
!mkdir lib && cd lib/

In [1]:
!ls lib

gcs-connector-hadoop3-2.2.5.jar
spark-3.1-bigquery-0.29.0-preview.jar
spark-3.2-bigquery-0.29.0-preview.jar
spark-bigquery-latest_2.12.jar
spark-bigquery-with-dependencies_2.13-0.29.0.jar


In [25]:
df = spark.read.parquet('/home/gbotemi/code/pq/2015/*/*/*')

In [21]:
df.show(3)

+----------+----------------+--------------------+--------------------+--------------------+------+-------------------+--------------------+
|        id|            type|               actor|                repo|             payload|public|         created_at|                 org|
+----------+----------------+--------------------+--------------------+--------------------+------+-------------------+--------------------+
|2489395767|       PushEvent|{https://avatars....|{28067809, soumit...|{null, 2d06657267...|  true|2015-01-01 01:00:00|                null|
|2489395768|       PushEvent|{https://avatars....|{20022094, radix/...|{null, d4c2903c26...|  true|2015-01-01 01:00:00|                null|
|2489395770|PullRequestEvent|{https://avatars....|{17165658, apache...|{closed, null, nu...|  true|2015-01-01 01:00:02|{https://avatars....|
+----------+----------------+--------------------+--------------------+--------------------+------+-------------------+--------------------+
only showing 

In [24]:
df.count()

199636

In [26]:
df.count()

2566901

In [42]:
df.actor.show()

TypeError: 'Column' object is not callable